In [3]:
import pandas as pd
import numpy as np
import hashlib

In [4]:
df = pd.read_excel("wwlLancMsc_data.xlsx")
df.head()

,site_national_code,site_description,site_local_code,Admission_Date,admission_date_dt,discharge_date_dt,spell_episode_los,specialty_local_code,specialty_spec_code,specialty_spec_desc,...,source_of_ref_description,presenting_complaint,acuity_code,inj_or_ail,NEWS2,attend_dis_description,ae_unplanned_attendance,location,ID,Deprivation Decile
0,RRF01,LEIGH INFIRMARY,LEI,2023-07-15,15/07/2023 13:57:00,15/07/2023 16:00:00,0,CREC,104,Colorectal Surgery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49591,3
1,RRF53,WRIGHTINGTON HOSPITAL,WNT,2023-07-05,2023-05-07 08:37:00,2023-05-07 10:08:00,0,TO,110,Trauma & Orthopaedics,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25760,10
2,RRF02,ROYAL ALBERT EDWARD INFIRMARY,RAEI,2023-06-08,2023-08-06 11:45:00,2023-08-06 12:50:00,0,GS,100,General Surgery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,159753,3
3,RRF53,WRIGHTINGTON HOSPITAL,WNT,2023-06-08,2023-08-06 07:13:00,2023-08-06 17:35:00,0,BS,103,Breast Surgery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76679,4
4,RRF02,ROYAL ALBERT EDWARD INFIRMARY,RAEI,2023-06-08,2023-08-06 08:29:00,2023-08-06 15:34:00,0,GA,301,Gastroenterology,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2617,9


In [6]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

In [7]:
ad = "admission_date_dt"
dis = "discharge_date_dt"

df[ad] = pd.to_datetime(df[ad], dayfirst=True, errors="coerce")
df[dis] = pd.to_datetime(df[dis], dayfirst=True, errors="coerce")

In [8]:
df["length_of_stay"] = (df[dis] - df[ad]).dt.total_seconds() / (24*3600)

In [9]:
df = df[df["length_of_stay"] >= 0]

los_95 = df["length_of_stay"].quantile(0.95)
df = df[df["length_of_stay"] <= los_95]


In [10]:
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df.select_dtypes(include=['object']).columns

df[num_cols] = df[num_cols].fillna(df[num_cols].median())

for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

In [11]:
if "id" in df.columns:
    df["patient_hash"] = df["id"].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest()[:12])
    df.drop(columns=["id"], inplace=True)
else:
    df["patient_hash"] = [hashlib.sha256(str(i).encode()).hexdigest()[:12] for i in range(len(df))]

C:\Users\Kumara Guru\AppData\Local\Temp\ipykernel_22340\2333217611.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["patient_hash"] = df["id"].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest()[:12])


In [12]:
age_col = None
for c in ["age","age_years"]:
    if c in df.columns:
        age_col = c

if age_col:
    df["age"] = pd.to_numeric(df[age_col], errors="coerce")
    df["age_group"] = pd.cut(df["age"],
                             bins=[0,18,40,60,80,120],
                             labels=["0-18","19-40","41-60","61-80","80+"])


In [13]:
print("Final Rows:", len(df))
print(df.isnull().sum().sort_values(ascending=False).head(10))
df.describe()


Final Rows: 37135
social_worker_date_time_referred        37135
discharge_delay_reason_national_code    37135
arrival_date                            28696
site_local_code                             0
site_national_code                          0
admission_date                              0
discharge_date_dt                           0
admission_date_dt                           0
specialty_spec_code                         0
spell_episode_los                           0
dtype: int64


,admission_date,admission_date_dt,discharge_date_dt,spell_episode_los,specialty_spec_code,duration_elective_wait,patient_age_on_admission,patient_age_on_discharge,discharge_delay_reason_national_code,delayed_discharges_flag,...,chronic_condition_cardiovascular_disease_flag,chronic_condition_hypertension_flag,chronic_condition_obesity_flag,chronic_condition_respiratory_flag,arrival_date,acuity_code,news2,ae_unplanned_attendance,deprivation_decile,length_of_stay
count,37135,37135,37135,37135.000000,37135.000000,37135.000000,37135.000000,37135.000000,0.0,37135.0,...,37135.000000,37135.000000,37135.000000,37135.000000,8439,37135.000000,37135.000000,37135.0,37135.000000,37135.000000
mean,2023-09-06 14:10:44.287060736,2023-08-12 16:10:51.861585920,2023-08-16 03:04:39.263767296,0.980504,242.072654,71.915013,56.354840,56.357587,NaN,0.0,...,0.393699,0.283775,0.120802,0.097105,2023-09-04 01:32:59.808033792,2.950451,1.122257,1.0,4.827333,3.454021
min,2023-06-08 00:00:00,2023-01-07 00:20:00,2023-01-07 00:59:00,0.000000,100.000000,0.000000,1.000000,1.000000,NaN,0.0,...,0.000000,0.000000,0.000000,0.000000,2023-06-07 00:00:00,1.000000,0.000000,1.0,1.000000,0.000694
25%,2023-07-24 00:00:00,2023-06-26 08:35:30,2023-06-28 13:56:30,0.000000,110.000000,47.000000,40.000000,40.000000,NaN,0.0,...,0.000000,0.000000,0.000000,0.000000,2023-07-21 00:00:00,3.000000,1.000000,1.0,2.000000,0.107639
50%,2023-09-07 00:00:00,2023-08-21 15:21:00,2023-08-23 21:20:00,0.000000,300.000000,47.000000,61.000000,61.000000,NaN,0.0,...,0.000000,0.000000,0.000000,0.000000,2023-09-05 00:00:00,3.000000,1.000000,1.0,4.000000,0.212500
75%,2023-10-23 00:00:00,2023-10-17 14:00:30,2023-10-19 19:43:30,0.000000,301.000000,47.000000,75.000000,75.000000,NaN,0.0,...,1.000000,1.000000,0.000000,0.000000,2023-10-18 00:00:00,3.000000,1.000000,1.0,8.000000,0.565972
max,2023-12-05 00:00:00,2023-12-11 22:03:00,2023-12-11 23:43:00,92.000000,560.000000,3415.000000,104.000000,104.000000,NaN,0.0,...,1.000000,1.000000,1.000000,1.000000,2023-12-05 00:00:00,4.000000,17.000000,1.0,10.000000,61.429861
std,NaN,NaN,NaN,3.668737,116.932865,119.296148,22.885444,22.886833,NaN,0.0,...,0.488576,0.450835,0.325902,0.296105,NaN,0.307569,1.008914,0.0,2.993867,10.388076


In [14]:
df.to_csv("NHS_Final_Cleaned.csv", index=False)


In [15]:
df.head()


,site_national_code,site_description,site_local_code,admission_date,admission_date_dt,discharge_date_dt,spell_episode_los,specialty_local_code,specialty_spec_code,specialty_spec_desc,...,presenting_complaint,acuity_code,inj_or_ail,news2,attend_dis_description,ae_unplanned_attendance,location,deprivation_decile,length_of_stay,patient_hash
0,RRF01,LEIGH INFIRMARY,LEI,2023-07-15,2023-07-15 13:57:00,2023-07-15 16:00:00,0,CREC,104,Colorectal Surgery,...,Unwell adult (GU/UA),3.0,Ailment,1.0,Admitted,1.0,ECC Awaiting Triage,3,0.085417,8e1eaa9c34c5
1,RRF53,WRIGHTINGTON HOSPITAL,WNT,2023-07-05,2023-05-07 08:37:00,2023-05-07 10:08:00,0,TO,110,Trauma & Orthopaedics,...,Unwell adult (GU/UA),3.0,Ailment,1.0,Admitted,1.0,ECC Awaiting Triage,10,0.063194,bcd60a7f499e
2,RRF02,ROYAL ALBERT EDWARD INFIRMARY,RAEI,2023-06-08,2023-08-06 11:45:00,2023-08-06 12:50:00,0,GS,100,General Surgery,...,Unwell adult (GU/UA),3.0,Ailment,1.0,Admitted,1.0,ECC Awaiting Triage,3,0.045139,3d14c2d4e4ce
3,RRF53,WRIGHTINGTON HOSPITAL,WNT,2023-06-08,2023-08-06 07:13:00,2023-08-06 17:35:00,0,BS,103,Breast Surgery,...,Unwell adult (GU/UA),3.0,Ailment,1.0,Admitted,1.0,ECC Awaiting Triage,4,0.431944,4cdc55453b39
4,RRF02,ROYAL ALBERT EDWARD INFIRMARY,RAEI,2023-06-08,2023-08-06 08:29:00,2023-08-06 15:34:00,0,GA,301,Gastroenterology,...,Unwell adult (GU/UA),3.0,Ailment,1.0,Admitted,1.0,ECC Awaiting Triage,9,0.295139,13d61412db18


In [16]:
df.shape


(37135, 102)

In [17]:
df.tail()


,site_national_code,site_description,site_local_code,admission_date,admission_date_dt,discharge_date_dt,spell_episode_los,specialty_local_code,specialty_spec_code,specialty_spec_desc,...,presenting_complaint,acuity_code,inj_or_ail,news2,attend_dis_description,ae_unplanned_attendance,location,deprivation_decile,length_of_stay,patient_hash
41841,RRF53,WRIGHTINGTON HOSPITAL,WNT,2023-12-05,2023-05-12 07:30:00,2023-05-12 15:19:00,0,TO,110,Trauma & Orthopaedics,...,Unwell adult (GU/UA),3.0,Ailment,1.0,Admitted,1.0,ECC Awaiting Triage,4,0.325694,f218bbcf0887
41842,RRF02,ROYAL ALBERT EDWARD INFIRMARY,RAEI,2023-12-05,2023-05-12 08:25:00,2023-05-12 12:46:00,0,GM,300,General Medicine,...,Shortness of breath in adults (SOB),2.0,Ailment,0.0,Discharged,1.0,ECC Awaiting Triage,2,0.181250,9370c8039bc6
41843,RRF01,LEIGH INFIRMARY,LEI,2023-12-05,2023-05-12 11:51:00,2023-05-12 18:09:00,0,UR,101,Urology,...,Unwell adult (GU/UA),3.0,Ailment,1.0,Admitted,1.0,ECC Awaiting Triage,6,0.262500,83c73e9c6f6d
41844,RRF53,WRIGHTINGTON HOSPITAL,WNT,2023-12-05,2023-05-12 06:58:00,2023-05-12 18:03:00,0,TO,110,Trauma & Orthopaedics,...,Unwell adult (GU/UA),3.0,Ailment,1.0,Admitted,1.0,ECC Awaiting Triage,3,0.461806,90e855751509
41845,RRF02,ROYAL ALBERT EDWARD INFIRMARY,RAEI,2023-12-05,2023-05-12 11:31:00,2023-05-12 17:14:00,0,GS,100,General Surgery,...,Unwell adult (GU/UA),3.0,Ailment,1.0,Admitted,1.0,ECC Awaiting Triage,1,0.238194,2be8c2ece07a


In [18]:
df.columns


Index(['site_national_code', 'site_description', 'site_local_code',
       'admission_date', 'admission_date_dt', 'discharge_date_dt',
       'spell_episode_los', 'specialty_local_code', 'specialty_spec_code',
       'specialty_spec_desc',
       ...
       'presenting_complaint', 'acuity_code', 'inj_or_ail', 'news2',
       'attend_dis_description', 'ae_unplanned_attendance', 'location',
       'deprivation_decile', 'length_of_stay', 'patient_hash'],
      dtype='object', length=102)

In [19]:
import os
os.listdir()


['.ipynb_checkpoints',
 'NHS_Final_Cleaned.csv',
 'scc450.ipynb',
 'Untitled.ipynb',
 'wwlLancMsc_data.xlsx']

In [20]:
cleaned_df = pd.read_csv("NHS_Final_Cleaned.csv")
cleaned_df.head()


,site_national_code,site_description,site_local_code,admission_date,admission_date_dt,discharge_date_dt,spell_episode_los,specialty_local_code,specialty_spec_code,specialty_spec_desc,...,presenting_complaint,acuity_code,inj_or_ail,news2,attend_dis_description,ae_unplanned_attendance,location,deprivation_decile,length_of_stay,patient_hash
0,RRF01,LEIGH INFIRMARY,LEI,2023-07-15,2023-07-15 13:57:00,2023-07-15 16:00:00,0,CREC,104,Colorectal Surgery,...,Unwell adult (GU/UA),3.0,Ailment,1.0,Admitted,1.0,ECC Awaiting Triage,3,0.085417,8e1eaa9c34c5
1,RRF53,WRIGHTINGTON HOSPITAL,WNT,2023-07-05,2023-05-07 08:37:00,2023-05-07 10:08:00,0,TO,110,Trauma & Orthopaedics,...,Unwell adult (GU/UA),3.0,Ailment,1.0,Admitted,1.0,ECC Awaiting Triage,10,0.063194,bcd60a7f499e
2,RRF02,ROYAL ALBERT EDWARD INFIRMARY,RAEI,2023-06-08,2023-08-06 11:45:00,2023-08-06 12:50:00,0,GS,100,General Surgery,...,Unwell adult (GU/UA),3.0,Ailment,1.0,Admitted,1.0,ECC Awaiting Triage,3,0.045139,3d14c2d4e4ce
3,RRF53,WRIGHTINGTON HOSPITAL,WNT,2023-06-08,2023-08-06 07:13:00,2023-08-06 17:35:00,0,BS,103,Breast Surgery,...,Unwell adult (GU/UA),3.0,Ailment,1.0,Admitted,1.0,ECC Awaiting Triage,4,0.431944,4cdc55453b39
4,RRF02,ROYAL ALBERT EDWARD INFIRMARY,RAEI,2023-06-08,2023-08-06 08:29:00,2023-08-06 15:34:00,0,GA,301,Gastroenterology,...,Unwell adult (GU/UA),3.0,Ailment,1.0,Admitted,1.0,ECC Awaiting Triage,9,0.295139,13d61412db18


In [21]:
# Extract mapping table
site_map = df[['site_national_code', 'site_description', 'site_local_code']].drop_duplicates()

print("Unique mapping rows:", len(site_map))
display(site_map.head())

Unique mapping rows: 4


,site_national_code,site_description,site_local_code
0,RRF01,LEIGH INFIRMARY,LEI
1,RRF53,WRIGHTINGTON HOSPITAL,WNT
2,RRF02,ROYAL ALBERT EDWARD INFIRMARY,RAEI
1908,RRF70,"BOSTON HOUSE, WIGAN HEALTH CTR",BOHS


In [22]:
df.drop(columns=['site_description', 'site_local_code'], inplace=True)


In [25]:
from sklearn.preprocessing import LabelEncoder

df_encoded = df.copy()

label_enc = LabelEncoder()

for col in df_encoded.select_dtypes(include=['object']).columns:
    df_encoded[col] = label_enc.fit_transform(df_encoded[col].astype(str))

df_encoded.head()


,site_national_code,admission_date,admission_date_dt,discharge_date_dt,spell_episode_los,specialty_local_code,specialty_spec_code,specialty_spec_desc,ward_code_admission,ward_name_admission,...,presenting_complaint,acuity_code,inj_or_ail,news2,attend_dis_description,ae_unplanned_attendance,location,deprivation_decile,length_of_stay,patient_hash
0,0,2023-07-15,2023-07-15 13:57:00,2023-07-15 16:00:00,0,6,104,4,17,17,...,43,3.0,0,1.0,0,1.0,0,3,0.085417,15223
1,2,2023-07-05,2023-05-07 08:37:00,2023-05-07 10:08:00,0,37,110,31,33,32,...,43,3.0,0,1.0,0,1.0,0,10,0.063194,20232
2,1,2023-06-08,2023-08-06 11:45:00,2023-08-06 12:50:00,0,13,100,11,42,43,...,43,3.0,0,1.0,0,1.0,0,3,0.045139,6567
3,2,2023-06-08,2023-08-06 07:13:00,2023-08-06 17:35:00,0,3,103,2,13,12,...,43,3.0,0,1.0,0,1.0,0,4,0.431944,8250
4,1,2023-06-08,2023-08-06 08:29:00,2023-08-06 15:34:00,0,9,301,9,16,16,...,43,3.0,0,1.0,0,1.0,0,9,0.295139,2090


In [26]:
df_encoded = pd.get_dummies(df_encoded, drop_first=True)
df_encoded.head()


,site_national_code,admission_date,admission_date_dt,discharge_date_dt,spell_episode_los,specialty_local_code,specialty_spec_code,specialty_spec_desc,ward_code_admission,ward_name_admission,...,presenting_complaint,acuity_code,inj_or_ail,news2,attend_dis_description,ae_unplanned_attendance,location,deprivation_decile,length_of_stay,patient_hash
0,0,2023-07-15,2023-07-15 13:57:00,2023-07-15 16:00:00,0,6,104,4,17,17,...,43,3.0,0,1.0,0,1.0,0,3,0.085417,15223
1,2,2023-07-05,2023-05-07 08:37:00,2023-05-07 10:08:00,0,37,110,31,33,32,...,43,3.0,0,1.0,0,1.0,0,10,0.063194,20232
2,1,2023-06-08,2023-08-06 11:45:00,2023-08-06 12:50:00,0,13,100,11,42,43,...,43,3.0,0,1.0,0,1.0,0,3,0.045139,6567
3,2,2023-06-08,2023-08-06 07:13:00,2023-08-06 17:35:00,0,3,103,2,13,12,...,43,3.0,0,1.0,0,1.0,0,4,0.431944,8250
4,1,2023-06-08,2023-08-06 08:29:00,2023-08-06 15:34:00,0,9,301,9,16,16,...,43,3.0,0,1.0,0,1.0,0,9,0.295139,2090


In [27]:
df_encoded.info()


<class 'pandas.core.frame.DataFrame'>
Index: 37135 entries, 0 to 41845
Data columns (total 100 columns):
 #   Column                                         Non-Null Count  Dtype         
---  ------                                         --------------  -----         
 0   site_national_code                             37135 non-null  int64         
 1   admission_date                                 37135 non-null  datetime64[ns]
 2   admission_date_dt                              37135 non-null  datetime64[ns]
 3   discharge_date_dt                              37135 non-null  datetime64[ns]
 4   spell_episode_los                              37135 non-null  int64         
 5   specialty_local_code                           37135 non-null  int64         
 6   specialty_spec_code                            37135 non-null  int64         
 7   specialty_spec_desc                            37135 non-null  int64         
 8   ward_code_admission                            37135 non-nul

In [28]:
df_encoded.to_csv("NHS_ML_Ready.csv", index=False)
